# Closed Captioning
Implements a feature using Langchain's image_captions.py and audio_speech_to_text.py to produce .srt files. This system will provide both subtitles and visual scene descriptions, essentially creating closed captioning.

## Imports

In [1]:
# imports for closed captioning
import cv2
import numpy as np
import os
import transformers
transformers.logging.set_verbosity_error()

from langchain.document_loaders import AssemblyAIAudioTranscriptLoader
from langchain.document_loaders import ImageCaptionLoader
from langchain.document_loaders.assemblyai import TranscriptFormat

## Object model

In [ ]:
class CaptionModel:
    def __init__(self, start_time, end_time, closed_caption):
        self.start_time = start_time
        self.end_time = end_time
        self.closed_caption = closed_caption

    def __str__(self):
        return f"start_time: {self.start_time}, end_time: {self.end_time}, closed_caption: {self.closed_caption}"

## Video model

In [5]:
class VideoModel:
    def __init__(self, start_time, end_time, image_description):
        self.start_time = start_time
        self.end_time = end_time
        self.image_description = image_description

    def __str__(self):
        return f"start_time: {self.start_time}, end_time: {self.end_time}, image_description: {self.image_description}"
    
    def get_start_time(self):
        return self.start_time
    
    def get_end_time(self):
        return self.end_time
    
    def get_image_description(self):
        return self.image_description
    
    def set_start_time(self, start_time):
        self.start_time = start_time

    def set_end_time(self, end_time):
        self.end_time = end_time
    
    def set_image_description(self, image_description):
        self.image_description = image_description

## Audio model

In [12]:
class AudioModel:
    def __init__(self, start_time, end_time, subtitle_text):
        self.start_time = start_time
        self.end_time = end_time
        self.subtitle_text = subtitle_text

    def __str__(self):
        return f"start_time: {self.start_time}, end_time: {self.end_time}, subtitle_text: {self.subtitle_text}"
    
    def get_start_time(self):
        return self.start_time
    
    def get_end_time(self):
        return self.end_time
    
    def get_subtitle_text(self):
        return self.subtitle_text
    
    def set_start_time(self, start_time):
        self.start_time = start_time

    def set_end_time(self, end_time):
        self.end_time = end_time
    
    def set_subtitle_text(self, subtitle_text):
        self.subtitle_text = subtitle_text

## Audio Speech to Text

In [ ]:
audio_file = "test_data/test.mp3"

loader = AssemblyAIAudioTranscriptLoader(
    file_path=audio_file, 
    api_key="f50c08e20ecd4544b175953636f0b936", 
    transcript_format=TranscriptFormat.SUBTITLES_SRT
)

docs = loader.load()

def CreateTranscriptModel(doc):
    transcription = doc.strip().split("\n")
    times = transcription[1].split(" --> ")
    start_time = times[0].strip()
    end_time = times[1].strip()

    subtitle_text = ' '.join(transcription[2:]).strip()

    transcript_model = AudioModel(start_time, end_time, subtitle_text)

    return transcript_model

print(CreateTranscriptModel(docs[0].page_content))

## Video Split to Frames

In [13]:
def frame_difference(prev_frame, curr_frame, threshold=30):
    # Compute the absolute difference between the current frame and the previous frame
    diff = cv2.absdiff(prev_frame, curr_frame)
    # Thresholding to get the binary image, where white represents significant difference
    _, thresh = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)
    # If there are any white pixels in thresh, the difference is significant
    return np.any(thresh)

# Initialize the video capture
capture = cv2.VideoCapture('test_data/video_test.mp4')
fps = capture.get(cv2.CAP_PROP_FPS)
frame_duration = 1000 / fps

video_models = []

frameNr = 0
ret, prev_frame = capture.read()
prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY) if ret else None
prev_start_time = 0
start_time = 0

while ret:
    end_time = prev_start_time
    prev_start_time = capture.get(cv2.CAP_PROP_POS_MSEC)

    ret, frame = capture.read()
    if not ret:
        start_time = end_time + frame_duration
        break
    
    start_time = prev_start_time
    # Convert to grayscale for comparison
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Compare with the previous frame
    if frameNr == 0 or frame_difference(prev_frame_gray, frame_gray):
        end_time = capture.get(cv2.CAP_PROP_POS_MSEC)
        cv2.imwrite(f'test_data/output_frames/frame.jpg', frame)
        prev_frame_gray = frame_gray

        # Define the path to the "output_frames" folder
        folder_path = f'test_data/output_frames/'

        # List all .jpg files in the folder
        image_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".jpg")]

        # Create an instance of the ImageCaptionLoader
        loader = ImageCaptionLoader(images=image_files)

        # Load captions for the images
        list_docs = loader.load()

        video_model = VideoModel(start_time, end_time, list_docs[len(list_docs) - 1].page_content)
        video_models.append(video_model)

        frameNr += 1

# Release the video capture object
capture.release()

In [ ]:
for video_model in video_models:
    print(video_model)